# Demo version of FuSL for surface data.

This demo version should serve as a template to run fusion searchlight (FuSL) on your own data. This pipline consists of two steps to prepare the data for FuSL. In the first step we load the datafiles and store it in a pandas dataframe (running the `load_fusl_cifti_data` function). In a next step we generate a numpy array and labels from this dataframe, which serve as inputs for FuSL (running the `prepare_fusl_data` function). There are three possibilities where you can enter your own data into this workflow:

- You can either structure and name your data files in the same manner as in this example in the `data` directory
- Or you can generate a pandas dataframe which has the same structure as `input_df`, including the matrix contaning searchlight neighborhoods (`neigh_adj`) (see `compute_neighborhoods.py`)
- Or you can generate a numpy array (`X`) and labels (`y`) containing your data, including an adjacency matrix contaning the fusion searchlight neighborhoods (`fusl_adj`)

The following script utilizes artificial data in the fsLR standard space in cifti format. You can alternatively run FuSL on your data in nifti format using the other demo (`FuSL_demo_vol.ipynb`) provided in this repository.

In [3]:
# Clone repository.
!git clone https://github.com/simonvino/constrained_ICA.git
%cd ./constrained_ICA/

Cloning into 'constrained_ICA'...
remote: Enumerating objects: 32, done.
remote: Counting objects: 100% (32/32), done.
remote: Compressing objects: 100% (30/30), done.
Receiving objects: 100% (32/32), 1.75 MiB | 1023.00 KiB/s, done.
remote: Total 32 (delta 14), reused 9 (delta 0), pack-reused 0 (from 0)
Resolving deltas: 100% (14/14), done.
/content/constrained_ICA


In [5]:
pip install hcp_utils nilearn mne

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.0/8.0 MB 10.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.5/10.5 MB 62.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 57.6 MB/s eta 0:00:00


In [6]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)
%cd /content/drive/'My Drive'/'Work'/FuSL

Mounted at /content/drive
/content/drive/My Drive/Work/FuSL


In [7]:
import hcp_utils as hcp
import numpy as np
from sklearn.svm import SVC
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.pipeline import make_pipeline
from nilearn import plotting

from utils.tfce_stats import tfce_stats
from fusl.fusion_search_light import fusion_search_light, compute_importance_maps
from utils.load_and_prepare import load_fusl_cifti_data, prepare_fusl_data

pixdim[1,2,3] should be non-zero; setting 0 dims to 1


## Load the data.
First we would like to load our datafiles and store it in a DataFrame, in which the different information sources are stored in the columns.
We can therefore use the `load_fusl_cifti_data` function, which assumes our datafiles have the following structure:

├── cifti <br>
│   ├── grp-1 <br>
│   │   ├── source-1_sub-01_ses-1_grp-1.dtseries.nii <br>
│   │   ├── source-1_sub-02_ses-1_grp-1.dtseries.nii <br>
│   │   ├── source-1_sub-03_ses-1_grp-1.dtseries.nii <br>
│   │   ├── ... <br>

├── grp-2 <br>
│   │   ├── source-1_sub-01_ses-1_grp-2.dtseries.nii <br>
│   │   ├── source-1_sub-02_ses-1_grp-2.dtseries.nii <br>
│   │   ├── source-1_sub-03_ses-1_grp-2.dtseries.nii <br>
│   │   ├── ... <br>

As an input we have to provide the path the the directory (`data_dir`), the names of the group directories (`groups`), and the names of the sources (`sources`). This function then additionally generates a matrix containing the neighborhoods of searchlights (`neigh_adj`) with a specified k-hop radius (`radius`). These neighborhoods are computed based on the adjacency matrix of the fsLR standard space provided in the [hcp_utils](https://rmldj.github.io/hcp-utils/) module.

In [8]:
# dataset = 'StN-1.0_ampstd-1_nsub-30_fullovlp'
# data_dir = os.path.join('../TSPO_pipe/data/fmri_surf/artificial_data/', dataset)
data_dir = './data/cifti/'
groups = ['grp-1', 'grp-2']
sources = ['source-1', 'source-2', 'source-3']

input_df, neigh_adj = load_fusl_cifti_data(data_dir=data_dir,
                                           groups=groups,
                                           sources=sources,
                                           radius=3,
                                           mask=hcp.struct.cortex_left,  # The artificial data is only defined on the left hemisphere for simplicity.
                                            verbose=False)

input_df.head(5)  # Visualize dataframe.

,sub,group,source-1,source-2,source-3
0,sub-01,grp-1,"[-2.859687985885645, 0.4242070257218908, 1.033...","[-1.1833440864488376, 0.012642424103859887, 0....","[-0.12019330704197674, 1.2277499666969507, -0...."
1,sub-02,grp-1,"[0.23259539685818717, -0.5459967427967656, -1....","[-0.24002694122168303, -0.1733079320764676, -0...","[2.2485238402814622, -0.20199574345892274, 1.2..."
2,sub-03,grp-1,"[-0.25573424866501515, -1.286018993540665, -0....","[0.5254885226391057, -0.3859018160286147, 0.40...","[-0.6126722612718664, -0.12771145150787572, 0...."
3,sub-04,grp-1,"[0.26224810072787286, -0.5109781616071851, 0.3...","[-0.9572861171314583, 0.4353117353486976, -0.0...","[1.1900669683794085, -1.0076451300483604, 0.75..."
4,sub-05,grp-1,"[-0.02961067294009472, 0.1712105685279312, 0.7...","[0.5122491301996641, 1.028562804529454, 0.1941...","[-1.1736969016965078, 0.49673350695671464, -0...."


## Format the data.

We then convert the data in our DataFrame `input_df` into a numpy array `X` and generate the group labels `y` to train the SL classifier. The features of different sources are concatenated in `X`, so `X` has the shape: (number_of_samples, number_of_sources * number_of_vertices). <br>
Also for each source we duplicate and horizontally concatenate our neighborhood adjacency matrix (`neigh_adj`), so that we get an adjacency matrix for our fusion searchlight (`fusl_adj`) with shape: (number_of_vertices, number_of_sources * number_of_vertices), where rows of `fusl_sl` contain SLs that simultaneously collect values from all sources.

In [9]:
# Specify sources used in FuSL.
sl_sources = ['source-1', 'source-2', 'source-3']

# Generate inputs for FuSL.
X, y, fusl_adj = prepare_fusl_data(input_df=input_df,
                                   sources=sl_sources,
                                   groups=groups,
                                   neigh_adj=neigh_adj)

Avgerage number of vertices within SLs: 60.5
X has shape: (60, 89088), FuSL adjacency matrix has shape: (29696, 89088).


## Run FuSL.
Now we have prepared our data and can set up our searchlight analysis.

In [10]:
import importlib, sys, time
importlib.reload(sys.modules['fusl.fusion_search_light'])
from fusl.fusion_search_light import fusion_search_light

In [11]:
# Define estimator.
clf = SVC(kernel='rbf')  # We can use any sci-kit learn classifier here.
estimator = make_pipeline(StandardScaler(), clf)  # Add standard scaling.

# Define cross-validation scheme.
cv = RepeatedStratifiedKFold(n_splits=10,
                             n_repeats=1,
                             random_state=42)

# Run search light.
start_time = time.time()
results = fusion_search_light(X=X,
                              y=y,
                              estimator=estimator,
                              A=fusl_adj,
                              cv=cv,
                              n_jobs=-1,
                              n_permutations=0,
                              shap=False)
end_time = time.time()
print('Finished after {:.2f} mins'.format((end_time - start_time)/60))  # 10000 perm: 6700 min, mamba: 2900 min, arti,100cvs: 130min

Finished after 27.41 mins


## Visualize results.
Once we finished running our analysis we can visualize the decoding accuracy and the impact of each source across the cortex. We first visualize the decoding accuracy here. Because the artificial data was defined only on the left hemisphere, we only use the meshes of the left cortex. We then threshold the decoding accuracy, using an arbitrary threshold (`acc_threshold`) of 75%. Usually we would here threshold the accuracy maps using permutation testing to find statistically significant accuracies, but because this is computationally more expensive we use in our demo at first an abitrary accuracy threshold.

In [12]:
# Define our meshes for plotting.
mesh_plt = hcp.mesh.inflated_left
slice_plt = hcp.left_cortex_data
bg_plt = hcp.mesh.sulc_left

acc_threshold = 0.75
avg_scores_thr = np.where(results['avg_scores'] > acc_threshold, results['avg_scores'], 0)

plotting.view_surf(mesh_plt,
                   slice_plt(avg_scores_thr),
                   threshold=1e-10,
                   cmap='bwr',
                   bg_map=bg_plt,
                   colorbar=True,
                   symmetric_cmap=False,
                   title='decoding accuracy')

Output hidden; open in https://colab.research.google.com to view.

If in our `fusion_search_light` function the computation of SHAP values was enabled (argument `shap=True`), we can then reconstruct the _importance_ of each individual input source. Our function output `results` contains then the mean absolute SHAP values of each searchlight (`results['shap_vals']`). We can average for each vertex these values across searchlights using the `compute_importance_maps` function. This generates for each source spatial maps of feature importance.

In [ ]:
# Re-run search light with shap=True.
start_time = time.time()
results = fusion_search_light(X=X,
                              y=y,
                              estimator=estimator,
                              A=fusl_adj,
                              cv=cv,
                              n_jobs=-1,
                              n_permutations=0,
                              shap=True)

In [ ]:
importance_maps = compute_importance_maps(shap_vals=results['shap_vals'],
                                         sources=sl_sources,
                                         neigh_adj=neigh_adj[vertices, vertices])

We can then weight these maps of feature _importance_ with the _informativeness_ at each vertex, by weighting the importance maps (`importance_maps`) with the thresholded decoding accuracy (`avg_scores_thr`).

In [ ]:
impact_maps = {}
for source in sel_sources:
    impact = avg_scores_thr * importance_maps[source]
    impact_maps.update({source: impact})

In [ ]:
plt_source = 'source-1'  # Select one source for plotting.

plotting.view_surf(mesh_plt,
                   slice_plt(impact_maps[plt_source]),
                   threshold=1e-10,
                   cmap='bwr',
                   vmax=0.025,
                   bg_map=bg_plt,
                   colorbar=True,
                   symmetric_cmap=True,
                   title='impact {}'.format(plt_source))

## Statistical testing.

To define, whether a decoding accuracy is statistically significant from chance, we can employ permutation testing. If `n_permutations > 0` the `fusion_search_light` function returns maps with accuracy scores, based on permuted labels (`results['perm_scores']`). <font color='red'>Note that this will be more time consuming!</font>

In [ ]:
n_permutations = 300

# Run search light again, with permutation testing enabled.
results = fusion_search_light(X=X,
                          y=y,
                          estimator=estimator,
                          A=fusl_adj,
                          cv=cv,
                          n_jobs=-1,
                          n_permutations=n_permutations,
                          shap=False)

To test for statistical significance and account for multiple comparisons across vertices we can utilize threshold-free cluster enhancement ([TFCE](https://www.fmrib.ox.ac.uk/datasets/techrep/tr08ss1/tr08ss1.pdf)) implemented in the `tfce_stats` function.

In [ ]:
# Compute tfce map and pvalues.
pvalues_tfce, scores_tfce = tfce_stats(scores=results['avg_scores'],
                                       permutation_scores=results['perm_scores'],
                                       adj=hcp.cortical_adjacency[mask, mask])

We can then use the (TFCE corrected) p-values to threshold our accuracy maps again.

In [ ]:
avg_scores_thr_tfce = np.where(pvalues_tfce < 0.05, results['avg_scores'], 0)

plotting.view_surf(mesh_plt,
                   slice_plt(avg_scores_thr_tfce),
                   threshold=1e-10,
                   cmap='bwr',
                   bg_map=bg_plt,
                   colorbar=True,
                   symmetric_cmap=False)

## Other stuff.

In [ ]:
!tree {dataset_dir} --filelimit 10